In [1]:
import sys
import os

# Add the build directory to Python path
build_dir = os.path.abspath('/Users/kisel/uni/apal/Algorithm-Selector/Implementation/lib')
sys.path.insert(0, build_dir)

# Add the data directory to Python path
data_dir = os.path.abspath('/Users/kisel/uni/apal/Algorithm-Selector/Implementation/data')
sys.path.insert(0, data_dir)

import pandas as pd
import numpy as np
from tqdm import tqdm

import sorters
import probes
import time
import pickle

import list_generators as lg
import runtime_features as rf
import create_features_dataset as cfd

with open('rf_model.pkl', 'rb') as f:
    rf_model = pickle.load(f)

In [15]:
def find_best_sorter(arr):
    best_name, best_time = None, float("inf")
    for sorter in sorters.list_sorters():
        time = sorters.benchmark_sorter(sorter, arr)
        if time < best_time:
            best_time, best_name = time, sorter

    return best_time, best_name

In [16]:
df = pd.read_feather(data_dir+"/out.feather")
df

,data
0,[]
1,"[0, 1, 2, 3, 4, 5, 6, 13, 8, 9, 44, 11, 12, 7,..."
2,"[47, 46, 45, 10, 43, 42, 41, 40, 39, 38, 37, 3..."
3,"[0, 1, 2, 3, 4, 5, 6, 7, 33, 9, 10, 11, 12, 13..."
4,"[47, 46, 45, 44, 43, 42, 24, 21, 39, 38, 37, 3..."
...,...
33512,"[1973, 1651, 3165, 1447, 4875, 3645, 6224, 134..."
33513,"[110, 4415, 7399, 8126, 7398, 8493, 8954, 1431..."
33514,"[8514, 1847, 4781, 6522, 4676, 4711, 149, 4041..."
33515,"[3597, 5973, 1450, 2313, 3993, 5910, 1979, 407..."


In [37]:
opt = []
for lst in tqdm(df['data'][20000:]):
    time, name = find_best_sorter(lst)
    opt.append((time, name))

100%|██████████| 13517/13517 [01:36<00:00, 140.56it/s] 


In [38]:
opt

[(0.249625, 'std_sort'),
 (0.230209, 'spin_sort'),
 (0.130584, 'std_sort'),
 (0.103125, 'std_sort'),
 (0.133334, 'std_sort'),
 (0.125208, 'std_sort'),
 (0.143709, 'std_sort'),
 (0.137542, 'std_sort'),
 (0.144542, 'spin_sort'),
 (0.133792, 'std_sort'),
 (0.155458, 'spin_sort'),
 (0.141084, 'std_sort'),
 (0.148166, 'spin_sort'),
 (0.138583, 'std_sort'),
 (0.15075, 'spin_sort'),
 (0.14525, 'std_sort'),
 (0.075459, 'quick_sort'),
 (0.088958, 'quick_sort'),
 (0.088291, 'quick_sort'),
 (0.107208, 'spin_sort'),
 (0.093917, 'std_sort'),
 (0.117125, 'spin_sort'),
 (0.110833, 'quick_sort'),
 (0.124833, 'spin_sort'),
 (0.133708, 'spin_sort'),
 (0.138083, 'spin_sort'),
 (0.13475, 'std_sort'),
 (0.146708, 'spin_sort'),
 (0.138709, 'std_sort'),
 (0.148458, 'spin_sort'),
 (0.146875, 'std_sort'),
 (0.154666, 'spin_sort'),
 (0.084292, 'quick_sort'),
 (0.081667, 'std_sort'),
 (0.109792, 'spin_sort'),
 (0.094083, 'std_sort'),
 (0.115458, 'spin_sort'),
 (0.112584, 'std_sort'),
 (0.125416, 'std_sort'),
 (0

In [39]:
times = 0
for entry in opt:
    times += entry[0]

print(times)

2379.4277369999795


In [61]:
features = []

for lst in tqdm(df['data'][30000:]):
    feature = rf.extract_features(lst)
    if feature:
        features.append(feature)
    else:
        print("Empty or None feature found:", lst)

features_df = pd.DataFrame(features)

100%|██████████| 3517/3517 [00:10<00:00, 333.75it/s] 


In [63]:
features_df = features_df[sorted(features_df.columns)]
print(features_df)

      avg_dup_distinct  avg_dup_pos  categorical_kurtosis  categorical_skew  \
0             0.666667     1.073118              1.154682          1.184986   
1             0.666667     1.073118              1.154682          1.184986   
2             0.818182     1.260215              1.241284          1.085653   
3             0.818182     1.260215              1.241284          1.085653   
4             0.818182     1.260215              1.241284          1.085653   
...                ...          ...                   ...               ...   
3512          0.333423     0.579497              2.950641          1.712855   
3513          0.052632     0.103000             19.257217          4.389503   
3514          0.052632     0.103000             19.257217          4.389503   
3515          0.176471     0.325800              5.701140          2.372116   
3516          0.666667     1.058400              1.771344          1.261681   

       dis    entropy  mono   range  runs   size  


In [65]:
guesses = rf_model.predict(features_df)
print(guesses)

['std_sort' 'std_sort' 'std_sort' ... 'std_sort' 'std_sort' 'std_sort']


In [ ]:
optimal_times = []
for sort in guesses:
    sorters.benchmark_sorter(sort)